# Textual Inversion

**Module 6.5, Lesson 3** | CourseAI

You know that CLIP's embedding table maps token IDs to 768-dimensional vectors. You know those vectors are learned parameters. You know CLIP's embedding space is continuous and meaningful. This notebook takes those three premises and makes textual inversion tangible---you will inspect the embedding table, verify gradient flow through frozen layers, train a real embedding, and compare it to LoRA.

**What you will do:**
- Inspect the CLIP embedding table, compute cosine similarities between tokens, and add a new pseudo-token to verify the table grows by one row
- Run one textual inversion training step by hand, verifying that gradients flow through the frozen pipeline to the embedding table, and that only the pseudo-token's row is actually *updated* (via the restore-original-rows pattern)
- Train a full textual inversion embedding on a concept dataset (~200--500 steps) and generate images with and without the learned pseudo-token
- Compare textual inversion vs LoRA on the same concept images: file size, training time, output quality

**For each exercise, PREDICT the output before running the cell.**

**Estimated time:** 45--60 minutes.

---

## Setup

Run this cell to install dependencies and import everything. This notebook requires a GPU (T4 or better) for training and inference.

In [ ]:
!pip install -q diffusers transformers accelerate peft datasets scikit-learn

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import gc
import time
import os

# Reproducible results
torch.manual_seed(42)
np.random.seed(42)

# Nice plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [10, 4]

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float16 if device.type == 'cuda' else torch.float32
print(f'Using device: {device}')
if device.type == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB')

print('\nSetup complete.')

## Shared Helpers

Display helpers, model ID, and VRAM management. Each exercise loads only the components it needs and cleans up afterward to stay within free-tier Colab VRAM (~16 GB on a T4).

> **VRAM tip:** If you encounter an out-of-memory error, go to Runtime -> Restart runtime and rerun from Setup. Exercises 3 and 4 use the most VRAM because they run training loops.

In [ ]:
model_id = 'stable-diffusion-v1-5/stable-diffusion-v1-5'


def show_images(images, titles, figsize=None):
    """Display a list of PIL images side by side."""
    n = len(images)
    if figsize is None:
        figsize = (5 * n, 5)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(np.array(img))
        ax.set_title(title, fontsize=10)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


def show_image_grid(images, titles, nrows, ncols, figsize=None, suptitle=None):
    """Display images in a grid with the given number of rows and columns."""
    if figsize is None:
        figsize = (4 * ncols, 4 * nrows)
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    axes_flat = axes.flat if nrows > 1 or ncols > 1 else [axes]
    for ax, img, title in zip(axes_flat, images, titles):
        ax.imshow(np.array(img))
        ax.set_title(title, fontsize=10)
        ax.axis('off')
    for ax in list(axes_flat)[len(images):]:
        ax.axis('off')
    if suptitle:
        plt.suptitle(suptitle, fontsize=13)
    plt.tight_layout()
    plt.show()


def cleanup():
    """Free GPU memory."""
    gc.collect()
    if device.type == 'cuda':
        torch.cuda.empty_cache()


print('Helpers defined.')

---

## Exercise 1: Inspect the CLIP Embedding Space [Guided]

The lesson explained that CLIP's vocabulary has 49,408 entries and that each entry maps a token ID to a 768-dimensional vector. You learned that these vectors are learned parameters (trainable rows in a matrix) and that CLIP's embedding space is continuous and meaningful---nearby vectors represent similar concepts.

This exercise makes the embedding table tangible. You will load CLIP's text model, inspect the embedding matrix dimensions, look up specific token embeddings, compute cosine similarities between related and unrelated tokens, and visualize a cluster of embeddings with PCA. Then you will add a new pseudo-token and verify the table grows by exactly one row.

**Before running, predict:**
- What shape will the embedding table have? (Think: how many tokens in CLIP's vocabulary, and what is the embedding dimension?)
- Will "cat" be closer to "kitten" or to "airplane" in cosine similarity? (Think: the "geometry encodes meaning" mental model from Embeddings & Position.)
- After adding a new token `<my-concept>`, what will the new embedding table shape be? (Think: one new row, same column dimension.)

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer

# Load CLIP's tokenizer and text encoder.
# We load in float32 for this exercise because we need precise
# cosine similarity computations (not training/inference).
print('Loading CLIP tokenizer and text encoder...')
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
text_encoder = CLIPTextModel.from_pretrained(
    model_id, subfolder='text_encoder', torch_dtype=torch.float32
).to(device)
print('Loaded.\n')

# ============================================================
# STEP 1: Inspect the embedding table dimensions.
# ============================================================
# The embedding table is the FIRST layer of the CLIP text encoder.
# It maps token IDs to dense vectors---the same nn.Embedding you
# built in Embeddings & Position.

embedding_layer = text_encoder.get_input_embeddings()
embedding_table = embedding_layer.weight.data

print(f'Embedding table shape: {list(embedding_table.shape)}')
print(f'  Vocabulary size:     {embedding_table.shape[0]} tokens')
print(f'  Embedding dimension: {embedding_table.shape[1]}')
print(f'  Total parameters:    {embedding_table.numel():,}')
print(f'\nThis is the lookup table from Embeddings & Position.')
print(f'Token ID i maps to row i of this matrix.')
print(f'Each row is a {embedding_table.shape[1]}-dimensional vector.')

In [ ]:
# ============================================================
# STEP 2: Look up embeddings for specific tokens.
# ============================================================
# Recall from Embeddings & Position: embedding.weight[i] gives you
# the same vector as embedding(tensor([i])). The embedding table
# is just a matrix, and lookup is just row selection.

words = ['cat', 'kitten', 'dog', 'puppy', 'airplane', 'mountain', 'sunset']
word_ids = {}
word_embeds = {}

for word in words:
    # Tokenize the word. CLIP's tokenizer may split into subword tokens,
    # but common words like these are single tokens.
    tokens = tokenizer.encode(word, add_special_tokens=False)
    token_id = tokens[0]
    word_ids[word] = token_id
    word_embeds[word] = embedding_table[token_id]
    print(f'  "{word}" -> token ID {token_id}, embedding shape: {list(word_embeds[word].shape)}')

print(f'\nEach word maps to a single row in the [{embedding_table.shape[0]}, {embedding_table.shape[1]}] table.')
print(f'These are the INITIAL embeddings---before the CLIP transformer contextualizes them.')

In [ ]:
# ============================================================
# STEP 3: Compute cosine similarities between token pairs.
# ============================================================
# The lesson reinforced the "geometry encodes meaning" mental model:
# nearby vectors represent similar concepts. Let's verify this
# with concrete numbers.

pairs = [
    ('cat', 'kitten'),     # very related (same animal, age variant)
    ('cat', 'dog'),        # related (both common pets)
    ('dog', 'puppy'),      # very related (same animal, age variant)
    ('cat', 'airplane'),   # unrelated
    ('mountain', 'sunset'),# somewhat related (both nature/landscape)
    ('airplane', 'sunset'),# weakly related at best
]

print('Cosine similarities between token embeddings:')
print('-' * 50)
similarities = []
for w1, w2 in pairs:
    e1 = word_embeds[w1].unsqueeze(0)
    e2 = word_embeds[w2].unsqueeze(0)
    cos_sim = F.cosine_similarity(e1, e2).item()
    similarities.append((w1, w2, cos_sim))
    print(f'  cos("{w1}", "{w2}") = {cos_sim:.4f}')

print(f'\n"Geometry encodes meaning." Related words have higher similarity.')
print(f'Cat-kitten and dog-puppy are the most similar pairs.')
print(f'Cat-airplane is the least similar.')
print(f'\nThis is the space where textual inversion will place a new point.')

In [ ]:
# ============================================================
# STEP 4: Visualize embeddings with PCA.
# ============================================================
# Project the 768-dim embeddings to 2D and see if semantic
# clusters are visible---even with just a few tokens.
# This connects to the EmbeddingSpaceExplorer from Embeddings & Position.

from sklearn.decomposition import PCA

# Gather a broader set of tokens for visualization.
viz_words = [
    'cat', 'kitten', 'tabby', 'fur',           # cat-related
    'dog', 'puppy', 'retriever',                # dog-related
    'airplane', 'helicopter', 'airport',        # aviation
    'mountain', 'sunset', 'ocean', 'forest',    # nature
    'painting', 'sketch', 'watercolor',         # art
]

viz_embeds = []
viz_labels = []
for word in viz_words:
    tokens = tokenizer.encode(word, add_special_tokens=False)
    embed = embedding_table[tokens[0]].cpu().numpy()
    viz_embeds.append(embed)
    viz_labels.append(word)

viz_matrix = np.stack(viz_embeds)

# PCA to 2D.
pca = PCA(n_components=2)
coords_2d = pca.fit_transform(viz_matrix)

# Color by category.
category_colors = {
    'cat': '#22c55e', 'kitten': '#22c55e', 'tabby': '#22c55e', 'fur': '#22c55e',
    'dog': '#3b82f6', 'puppy': '#3b82f6', 'retriever': '#3b82f6',
    'airplane': '#ef4444', 'helicopter': '#ef4444', 'airport': '#ef4444',
    'mountain': '#f59e0b', 'sunset': '#f59e0b', 'ocean': '#f59e0b', 'forest': '#f59e0b',
    'painting': '#a855f7', 'sketch': '#a855f7', 'watercolor': '#a855f7',
}

fig, ax = plt.subplots(figsize=(10, 8))
for i, word in enumerate(viz_labels):
    color = category_colors.get(word, '#ffffff')
    ax.scatter(coords_2d[i, 0], coords_2d[i, 1], c=color, s=80, zorder=5)
    ax.annotate(word, (coords_2d[i, 0], coords_2d[i, 1]),
                textcoords='offset points', xytext=(8, 5),
                fontsize=10, color=color)

ax.set_title('CLIP Token Embeddings Projected to 2D (PCA)', fontsize=12)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=10)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=10)
ax.grid(alpha=0.2)
plt.tight_layout()
plt.show()

print('Even in a 2D projection of 768-dim space, semantic clusters are visible.')
print('Cat-related tokens cluster together. Aviation tokens cluster together.')
print('Textual inversion will find a NEW point in this space---not at any')
print('existing token\'s position, but in the continuous space between them.')

In [ ]:
# ============================================================
# STEP 5: Add a new pseudo-token and verify the table grows.
# ============================================================
# This is the mechanical heart of textual inversion: extend the
# vocabulary by one token, resize the embedding table by one row.

original_vocab_size = len(tokenizer)
original_table_shape = list(text_encoder.get_input_embeddings().weight.shape)

print(f'BEFORE adding token:')
print(f'  Vocabulary size:     {original_vocab_size}')
print(f'  Embedding table:     {original_table_shape}')

# Add the pseudo-token to the tokenizer.
num_added = tokenizer.add_tokens(['<my-concept>'])
new_token_id = tokenizer.convert_tokens_to_ids('<my-concept>')

# Resize the text encoder's embedding table to accommodate the new token.
# This adds one row of randomly initialized values.
text_encoder.resize_token_embeddings(len(tokenizer))

new_vocab_size = len(tokenizer)
new_table_shape = list(text_encoder.get_input_embeddings().weight.shape)

print(f'\nAFTER adding token:')
print(f'  Vocabulary size:     {new_vocab_size}')
print(f'  Embedding table:     {new_table_shape}')
print(f'  New token "<my-concept>" -> ID {new_token_id}')
print(f'  Tokens added:        {num_added}')

# Inspect the new row.
new_embed = text_encoder.get_input_embeddings().weight.data[new_token_id]
print(f'\nNew embedding row shape: {list(new_embed.shape)}')
print(f'New embedding (first 5 values): {new_embed[:5].tolist()}')
print(f'These are random values. The embedding has no meaning yet.')
print(f'\nTextual inversion will optimize this one row---768 floats---so that')
print(f'prompts containing <my-concept> generate images of a specific concept.')

In [ ]:
# ============================================================
# STEP 6: Initialize from a related word (optional strategy).
# ============================================================
# Instead of starting from a random point, start from "cat".
# This places the optimization in the right neighborhood.

cat_id = tokenizer.convert_tokens_to_ids('cat')
cat_embed = text_encoder.get_input_embeddings().weight.data[cat_id].clone()

# Overwrite the random embedding with the "cat" embedding.
text_encoder.get_input_embeddings().weight.data[new_token_id] = cat_embed

# Verify the initialization.
initialized_embed = text_encoder.get_input_embeddings().weight.data[new_token_id]
cos_sim = F.cosine_similarity(
    initialized_embed.unsqueeze(0),
    cat_embed.unsqueeze(0),
).item()

print(f'Initialized <my-concept> from "cat" embedding.')
print(f'Cosine similarity between <my-concept> and "cat": {cos_sim:.4f}')
print(f'(Should be 1.0---they are identical before training.)')
print(f'\nInitializing from a related word is the "geometry encodes meaning"')
print(f'mental model applied to optimization: start in the right neighborhood,')
print(f'and the optimizer has less distance to travel.')

In [ ]:
# Clean up before Exercise 2.
del text_encoder, tokenizer, embedding_table
cleanup()
print('CLIP text encoder freed from VRAM.')

### What Just Happened

You inspected CLIP's embedding table as a concrete data structure and verified three things from the lesson:

1. **The embedding table is a matrix.** Shape [49408, 768]---49,408 tokens, each mapped to a 768-dim vector. This is the same `nn.Embedding` you built in Embeddings & Position, just larger.

2. **Geometry encodes meaning.** Cosine similarities confirmed that related tokens (cat/kitten, dog/puppy) are closer in embedding space than unrelated tokens (cat/airplane). The PCA visualization showed semantic clusters even in 2D projection.

3. **Adding a new token is mechanically simple.** `tokenizer.add_tokens()` extends the vocabulary, `resize_token_embeddings()` adds one row to the embedding matrix. The new row starts with random values (or can be initialized from a related word). That one row---768 floats---is the entire optimization target of textual inversion.

---

## Exercise 2: One Textual Inversion Training Step by Hand [Guided]

The lesson showed the textual inversion training loop as a side-by-side comparison with the LoRA training loop. The steps are identical---VAE encode, sample timestep, noise, U-Net prediction, MSE loss, backprop. The only difference: what receives the gradient update. In LoRA, it was the adapter matrices. Here, it is a single embedding row.

This exercise runs one training step from scratch. You will create a pseudo-token embedding, construct a prompt containing it, run the full forward pass (tokenizer -> embedding lookup -> CLIP text encoder -> U-Net noise prediction -> MSE loss), then backprop and inspect the gradients.

**Important subtlety:** The embedding table is a single tensor. When we enable `requires_grad` on it, *all* rows that participate in the forward pass receive gradients through CLIP's self-attention (every token interacts with every other token). The trick in textual inversion is not that only one row *gets* gradients---it is that only one row *gets updated*. We achieve this by saving the original embeddings and restoring them after each optimizer step, so only the pseudo-token's row actually changes.

**Before running, predict:**
- Will the embedding rows for tokens like "a", "photo", "of" also receive non-zero gradients? (Think: CLIP's transformer applies self-attention across all 77 tokens.)
- Will the CLIP text encoder's transformer weights have gradients? (Think: they are frozen with `requires_grad=False`, but gradients flow THROUGH them to reach the embedding.)
- What pattern ensures only the pseudo-token's row is actually updated? (Think: save the original rows, restore them after each optimizer step.)

In [ ]:
from diffusers import UNet2DConditionModel, AutoencoderKL
from transformers import CLIPTextModel, CLIPTokenizer

# Load all pipeline components.
# We use float32 for this exercise because we need to inspect gradients
# precisely. In Exercises 3 and 4, we will switch to float16 for speed.
print('Loading pipeline components in float32...')
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
text_encoder = CLIPTextModel.from_pretrained(
    model_id, subfolder='text_encoder', torch_dtype=torch.float32
).to(device)
vae = AutoencoderKL.from_pretrained(
    model_id, subfolder='vae', torch_dtype=torch.float32
).to(device)
unet = UNet2DConditionModel.from_pretrained(
    model_id, subfolder='unet', torch_dtype=torch.float32
).to(device)

print('Components loaded.')

In [ ]:
# ============================================================
# STEP 1: Create the pseudo-token and set up freeze/unfreeze.
# ============================================================
# This is the 5-step setup from the lesson pseudocode:
#   1. Add token to tokenizer
#   2. Resize embedding table
#   3. Initialize from a related word
#   4. Freeze EVERYTHING
#   5. Unfreeze ONLY the new embedding row

# 1. Add the pseudo-token.
placeholder_token = '<my-concept>'
tokenizer.add_tokens([placeholder_token])
placeholder_token_id = tokenizer.convert_tokens_to_ids(placeholder_token)

# 2. Resize the embedding table.
text_encoder.resize_token_embeddings(len(tokenizer))

# 3. Initialize from "cat" (a related word).
token_embeds = text_encoder.get_input_embeddings().weight
cat_id = tokenizer.convert_tokens_to_ids('cat')
with torch.no_grad():
    token_embeds[placeholder_token_id] = token_embeds[cat_id].clone()

# 4. Freeze EVERYTHING.
vae.requires_grad_(False)
unet.requires_grad_(False)
text_encoder.requires_grad_(False)

# 5. Unfreeze ONLY the new embedding row.
# The embedding weight is the whole table; we need to enable grad for it
# and then mask the update in the optimizer. For verification purposes,
# we enable requires_grad on the entire embedding weight (so gradients
# are computed for the new row), but we will check that gradients are
# effectively zero for all other rows.
token_embeds.requires_grad_(True)

print(f'Pseudo-token: "{placeholder_token}" -> ID {placeholder_token_id}')
print(f'Embedding table shape: {list(token_embeds.shape)}')
print(f'\nFrozen:')
print(f'  VAE:          {sum(1 for p in vae.parameters() if not p.requires_grad)} params frozen')
print(f'  U-Net:        {sum(1 for p in unet.parameters() if not p.requires_grad)} params frozen')
print(f'  Text encoder: all weights frozen except embedding table')
print(f'\nTrainable: embedding table ({token_embeds.numel():,} floats total,')
print(f'  but only row {placeholder_token_id} will be optimized = 768 floats)')

In [ ]:
# ============================================================
# STEP 2: Create a synthetic training image.
# ============================================================
# For this single-step exercise we do not need a real dataset.
# A random image is sufficient---we just want to verify gradient flow.

# Create a random "training image" tensor in [-1, 1].
torch.manual_seed(42)
fake_image = torch.randn(1, 3, 512, 512, device=device, dtype=torch.float32)

# VAE encode the image to latent space (frozen).
with torch.no_grad():
    latent_dist = vae.encode(fake_image)
    z_0 = latent_dist.latent_dist.mode() * vae.config.scaling_factor

print(f'Fake training image shape:  {list(fake_image.shape)}')
print(f'VAE-encoded latent z_0:     {list(z_0.shape)}')
print(f'\nThe VAE encoder converts the image to latent space. Frozen---no gradients.')

In [ ]:
# ============================================================
# STEP 3: Construct and tokenize the prompt.
# ============================================================
# The prompt contains the pseudo-token. The tokenizer converts it
# to an integer ID, which the embedding table maps to a vector.

prompt = f'a photo of {placeholder_token}'

text_input = tokenizer(
    prompt,
    padding='max_length',
    max_length=tokenizer.model_max_length,
    truncation=True,
    return_tensors='pt',
)
input_ids = text_input.input_ids.to(device)

print(f'Prompt: "{prompt}"')
print(f'Token IDs: {input_ids[0, :10].tolist()}... (padded to {input_ids.shape[1]})')
print(f'\nToken ID {placeholder_token_id} is the pseudo-token <my-concept>.')
print(f'The tokenizer treats it like any other token---it is just a new entry')
print(f'in the vocabulary.')

In [ ]:
# ============================================================
# STEP 4: Forward pass through CLIP text encoder.
# ============================================================
# This is the two-stage pipeline from the lesson:
#   Stage 1: Token IDs -> embedding lookup (our trainable row is here)
#   Stage 2: Embeddings -> CLIP transformer -> contextual embeddings
#
# We need to enable gradients for this pass even though the CLIP
# transformer is frozen, because gradients must flow THROUGH the
# frozen layers to reach the embedding table.

# Forward pass with gradients enabled (no torch.no_grad!).
encoder_output = text_encoder(input_ids)
text_embeddings = encoder_output[0]  # [1, 77, 768]

print(f'Text embeddings shape: {list(text_embeddings.shape)}')
print(f'  Batch size:          {text_embeddings.shape[0]}')
print(f'  Sequence length:     {text_embeddings.shape[1]} (padded to 77)')
print(f'  Embedding dimension: {text_embeddings.shape[2]}')
print(f'\nThese are CONTEXTUAL embeddings---the CLIP transformer has applied')
print(f'self-attention across all tokens. The pseudo-token\'s embedding has')
print(f'been contextualized with "a", "photo", "of", and the padding tokens.')

In [ ]:
# ============================================================
# STEP 5: The DDPM training step (sample t, noise, predict, loss).
# ============================================================
# This is the same training loop from Learning to Denoise and LoRA
# Fine-Tuning. The only difference: what receives the gradient.

from diffusers import DDPMScheduler

noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder='scheduler')

# Sample a random timestep.
torch.manual_seed(42)
timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (1,), device=device)

# Sample random noise.
noise = torch.randn_like(z_0)

# Add noise to the latent using the forward process formula:
# z_t = sqrt(alpha_bar_t) * z_0 + sqrt(1 - alpha_bar_t) * epsilon
noisy_latents = noise_scheduler.add_noise(z_0, noise, timesteps)

# U-Net predicts the noise (frozen weights, but gradients flow through).
noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=text_embeddings).sample

# MSE loss between predicted noise and actual noise.
loss = F.mse_loss(noise_pred, noise)

print(f'Timestep t:         {timesteps.item()}')
print(f'Noisy latent shape: {list(noisy_latents.shape)}')
print(f'Noise pred shape:   {list(noise_pred.shape)}')
print(f'MSE loss:           {loss.item():.6f}')
print(f'\nThis is the same DDPM training step from Learning to Denoise.')
print(f'Same formula, same loss, same pipeline. The only difference is next.')

In [ ]:
# ============================================================
# STEP 6: Backprop and inspect gradient flow.
# ============================================================
# Gradients flow backward through:
#   loss -> U-Net (frozen) -> cross-attention (frozen) ->
#   CLIP transformer (frozen) -> embedding lookup
#
# IMPORTANT: Because the embedding table is ONE tensor with
# requires_grad=True, ALL rows that participated in the forward
# pass receive gradients (via CLIP's self-attention, every token
# interacts with every other). The textual inversion trick is
# not that only one row GETS gradients---it is that only one
# row GETS UPDATED (via the restore-original-rows pattern).

loss.backward()

# Check the embedding table's gradient.
embed_grad = token_embeds.grad

print(f'Embedding table gradient shape: {list(embed_grad.shape)}')
print(f'  (Same shape as the embedding table itself)')

# Check which rows have non-zero gradients.
row_grad_norms = embed_grad.norm(dim=1)  # L2 norm per row
nonzero_rows = (row_grad_norms > 0).nonzero(as_tuple=True)[0]

print(f'\nRows with non-zero gradients: {len(nonzero_rows)} rows')

# Show gradients for all tokens in the prompt.
prompt_token_ids = input_ids[0].tolist()
unique_prompt_ids = sorted(set(prompt_token_ids))

print(f'\nGradient norms for tokens used in the prompt:')
for tid in unique_prompt_ids:
    token_str = tokenizer.decode([tid])
    norm = row_grad_norms[tid].item()
    marker = '*** PSEUDO-TOKEN ***' if tid == placeholder_token_id else ''
    print(f'  Token {tid:>5d} "{token_str:>12s}": gradient norm = {norm:.6f}  {marker}')

print(f'\nNotice: MULTIPLE rows have non-zero gradients, not just the pseudo-token.')
print(f'This is because CLIP\'s self-attention means every token interacts with')
print(f'every other token. Gradients flow back to ALL participating embeddings.')
print(f'\nSo how does textual inversion update ONLY the pseudo-token? See next cell.')

# ============================================================
# STEP 6b: Demonstrate the restore-original-rows pattern.
# ============================================================
# This is the actual mechanism used in textual inversion training:
# after each optimizer step, restore all rows except the pseudo-token's.

print(f'\n{"=" * 55}')
print(f'The restore-original-rows pattern:')
print(f'{"=" * 55}')

# Save the current state of the pseudo-token row before update.
original_pseudo_embed = token_embeds.data[placeholder_token_id].clone()

# Save original embeddings for all OTHER rows.
original_other_embeds = token_embeds.data[:placeholder_token_id].clone()

# Simulate an optimizer step (this would update ALL rows with gradients).
simple_optimizer = torch.optim.AdamW([token_embeds], lr=5e-4)
simple_optimizer.step()

# Check: the pseudo-token row has changed.
pseudo_changed = not torch.equal(token_embeds.data[placeholder_token_id], original_pseudo_embed)
print(f'\nAfter optimizer.step():')
print(f'  Pseudo-token row changed: {pseudo_changed}')

# Now restore the original rows (this is the key trick).
with torch.no_grad():
    token_embeds.data[:placeholder_token_id] = original_other_embeds

# Verify: a prompt-participating token (e.g., "a") is back to its original value.
a_token_id = tokenizer.convert_tokens_to_ids('a</w>')
a_restored = torch.equal(token_embeds.data[:placeholder_token_id], original_other_embeds)
print(f'  Other rows restored to original: {a_restored}')
print(f'\nThis is how textual inversion ensures only one row is updated:')
print(f'  1. Enable requires_grad on the full embedding table')
print(f'  2. Run the optimizer step (updates all rows with gradients)')
print(f'  3. Restore all rows except the pseudo-token to their originals')
print(f'\nThe result: only the pseudo-token\'s embedding actually changes.')

In [ ]:
# ============================================================
# STEP 7: Verify frozen model weights have zero weight updates.
# ============================================================
# The CLIP transformer and U-Net are frozen---their weights should
# have NO gradients (requires_grad=False).

print('Checking frozen model weights for gradients...')
print()

# Check U-Net.
unet_grads = sum(1 for p in unet.parameters() if p.grad is not None and p.grad.abs().sum() > 0)
print(f'U-Net parameters with non-zero gradients: {unet_grads}')

# Check VAE.
vae_grads = sum(1 for p in vae.parameters() if p.grad is not None and p.grad.abs().sum() > 0)
print(f'VAE parameters with non-zero gradients:   {vae_grads}')

# Check CLIP text encoder (excluding the embedding table, which we unfroze).
clip_grads = 0
for name, p in text_encoder.named_parameters():
    if 'token_embedding' in name:
        continue  # Skip the embedding table---we unfroze it
    if p.grad is not None and p.grad.abs().sum() > 0:
        clip_grads += 1
print(f'CLIP encoder weights with non-zero gradients (excluding embedding): {clip_grads}')

print(f'\nAll zeros. The gradients flowed THROUGH the frozen CLIP encoder and')
print(f'U-Net to reach the embedding table, but no frozen weights were updated.')
print(f'This is exactly the mechanism from the lesson: the frozen layers are a')
print(f'pathway, not a destination. Gradients pass through to reach the one')
print(f'trainable parameter at the very start of the pipeline.')

print(f'\n=== Summary ===')
print(f'Trainable parameters:  {token_embeds.shape[1]} (one embedding row)')
print(f'Frozen parameters:     ~{sum(p.numel() for p in unet.parameters()) / 1e6:.0f}M (U-Net) + ')
print(f'                       ~{sum(p.numel() for p in text_encoder.parameters()) / 1e6:.0f}M (CLIP) + ')
print(f'                       ~{sum(p.numel() for p in vae.parameters()) / 1e6:.0f}M (VAE)')
print(f'\nOne embedding row of 768 floats. That is the entire "model" you train.')

In [ ]:
# Clean up before Exercise 3.
del text_encoder, tokenizer, vae, unet, z_0, noise_pred, loss, embed_grad
cleanup()
print('All components freed from VRAM.')

### What Just Happened

You ran one complete textual inversion training step by hand and verified the core mechanism:

1. **The training step is identical to DDPM/LoRA training.** VAE encode, sample timestep, noise, U-Net prediction, MSE loss. The same loop you have seen three times before.

2. **Multiple embedding rows receive gradients, not just the pseudo-token.** Because CLIP's transformer applies self-attention across all 77 tokens, gradients flow back to every embedding row that participated in the forward pass. This is a consequence of the embedding table being a single tensor with `requires_grad=True`.

3. **Only the pseudo-token's row is actually *updated*.** The restore-original-rows pattern ensures this: after each optimizer step, all rows except the pseudo-token's are restored to their original values. The mechanism is simple but essential---it is how 768 parameters are isolated within a 49,409 x 768 table.

4. **Frozen model weights have zero gradient updates.** The U-Net, VAE, and CLIP transformer weights are frozen (`requires_grad=False`), so they accumulate no gradients at all. Gradients flow *through* these frozen layers to reach the embedding table, but the frozen layers themselves are not updated. The CLIP encoder is a frozen pathway, not a wall.

5. **768 trainable parameters (effectively).** One embedding row, protected by the restore pattern. Compare to LoRA's ~200K--400K parameters, or full U-Net fine-tuning's ~860M. Three orders of magnitude smaller than LoRA.

---

## Exercise 3: Train Textual Inversion on a Concept Dataset [Supported]

You verified the mechanism in Exercise 2 with a single step and a fake image. Now you will train a real textual inversion embedding on a small concept dataset. We use `lambdalabs/naruto-blip-captions` (the same dataset used in the LoRA lesson) as our training data---it provides a consistent visual style that we can attempt to capture.

**Deliberate choice:** The lesson explained that textual inversion works best for specific visual objects (like "my cat") and struggles with complex styles. We are *intentionally* training on a style dataset here to give you firsthand experience of that limitation. Notice how the results compare to what LoRA achieved on the same dataset---this sets up Exercise 4's direct comparison.

We train for 200--500 steps (shortened from the paper's 3,000--5,000 for Colab time constraints). You will generate images at training checkpoints to observe convergence, then compare generation with and without the learned pseudo-token.

**Your tasks:**
- Complete the training loop (fill in the TODO sections)
- Generate images at checkpoints to observe convergence
- Generate with and without the pseudo-token after training

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, DPMSolverMultistepScheduler, StableDiffusionPipeline
from datasets import load_dataset
from torchvision import transforms

# Load all components in float32 (needed for stable training gradients).
print('Loading pipeline components...')
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
text_encoder = CLIPTextModel.from_pretrained(
    model_id, subfolder='text_encoder', torch_dtype=torch.float32
).to(device)
vae = AutoencoderKL.from_pretrained(
    model_id, subfolder='vae', torch_dtype=torch.float32
).to(device)
unet = UNet2DConditionModel.from_pretrained(
    model_id, subfolder='unet', torch_dtype=torch.float32
).to(device)
noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder='scheduler')

print('Loading dataset...')
dataset = load_dataset('lambdalabs/naruto-blip-captions', split='train')
print(f'Dataset size: {len(dataset)} images')
print('Components loaded.')

In [ ]:
# ============================================================
# Set up the pseudo-token (same 5 steps as Exercise 2).
# ============================================================

placeholder_token = '<naruto-style>'
initializer_token = 'anime'  # Initialize from a semantically relevant word

# 1. Add the pseudo-token to the tokenizer.
tokenizer.add_tokens([placeholder_token])
placeholder_token_id = tokenizer.convert_tokens_to_ids(placeholder_token)

# 2. Resize the embedding table.
text_encoder.resize_token_embeddings(len(tokenizer))

# 3. Initialize from the semantically relevant word.
token_embeds = text_encoder.get_input_embeddings().weight
init_id = tokenizer.convert_tokens_to_ids(initializer_token)
with torch.no_grad():
    token_embeds[placeholder_token_id] = token_embeds[init_id].clone()

# 4. Freeze EVERYTHING.
vae.requires_grad_(False)
unet.requires_grad_(False)
text_encoder.requires_grad_(False)

# 5. Unfreeze the embedding table so gradients can reach the new row.
text_encoder.get_input_embeddings().weight.requires_grad_(True)

print(f'Pseudo-token: "{placeholder_token}" -> ID {placeholder_token_id}')
print(f'Initialized from: "{initializer_token}" (ID {init_id})')
print(f'Embedding dimension: {token_embeds.shape[1]}')

In [ ]:
# ============================================================
# Prepare the dataset and training utilities.
# ============================================================

image_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),  # Scale to [-1, 1]
])

# The prompt template replaces the original caption with our pseudo-token.
# This is the standard textual inversion approach: associate the pseudo-token
# with the visual content of the training images.
prompt_template = f'a painting in the style of {placeholder_token}'


def get_training_batch(dataset, batch_idx, batch_size=1):
    """Get a batch of (image_tensor, tokenized_prompt) from the dataset."""
    idx = batch_idx % len(dataset)
    example = dataset[idx]
    image = example['image'].convert('RGB')
    image_tensor = image_transform(image).unsqueeze(0).to(device)

    text_input = tokenizer(
        prompt_template,
        padding='max_length',
        max_length=tokenizer.model_max_length,
        truncation=True,
        return_tensors='pt',
    )
    input_ids = text_input.input_ids.to(device)

    return image_tensor, input_ids


print(f'Prompt template: "{prompt_template}"')
print(f'Image resolution: 512x512')
print('Training utilities ready.')

In [ ]:
# ============================================================
# Training loop.
# ============================================================
# This is the same DDPM training loop from Learning to Denoise
# and LoRA Fine-Tuning. The only change: the optimizer targets
# a single embedding row instead of LoRA adapter matrices.

num_training_steps = 300
learning_rate = 5e-3  # Higher LR than LoRA because we optimize fewer params
checkpoint_steps = [50, 100, 200, 300]  # Steps at which to save embeddings

# The optimizer targets only the embedding table.
# In a production implementation, you would create a custom optimizer
# that masks updates to all rows except the new one. Here, we use a
# simpler approach: after each step, we restore the original embeddings
# for all rows except the pseudo-token's row.

# Save the original embeddings so we can restore them.
original_embeds = token_embeds.data[:placeholder_token_id].clone()

# TODO 1: Create the optimizer.
# The optimizer should target text_encoder.get_input_embeddings().weight
# with the learning rate defined above. Use torch.optim.AdamW.
#
# optimizer = torch.optim.AdamW(
#     [text_encoder.get_input_embeddings().weight],
#     lr=learning_rate,
# )
optimizer = None  # Replace this line

# Storage for loss history and checkpoint embeddings.
loss_history = []
checkpoint_embeds = {}

print(f'Training for {num_training_steps} steps...')
print(f'Learning rate: {learning_rate}')
print(f'Checkpoints at steps: {checkpoint_steps}')

start_time = time.time()

for step in range(num_training_steps):
    # Get a training batch.
    image_tensor, input_ids = get_training_batch(dataset, step)

    # TODO 2: Implement one DDPM training step.
    # This is the same 5-step pattern from Exercise 2 and the LoRA lesson:
    #   a) VAE encode the image (with torch.no_grad)
    #   b) Sample a random timestep
    #   c) Sample noise and add it to the latent (noise_scheduler.add_noise)
    #   d) Forward pass through text_encoder and unet
    #   e) Compute MSE loss between predicted noise and actual noise
    #
    # Hints:
    #   - VAE encode: vae.encode(image_tensor).latent_dist.mode() * vae.config.scaling_factor
    #   - Random timestep: torch.randint(0, noise_scheduler.config.num_train_timesteps, (1,), device=device)
    #   - Add noise: noise_scheduler.add_noise(latents, noise, timesteps)
    #   - Text encoder: text_encoder(input_ids)[0]  (no torch.no_grad---gradients must flow!)
    #   - U-Net: unet(noisy_latents, timesteps, encoder_hidden_states=text_emb).sample
    #   - Loss: F.mse_loss(noise_pred, noise)

    loss = None  # Replace with actual loss computation

    if loss is None:
        print('Fill in TODO 2 to run the training loop.')
        break

    # Backprop and optimizer step.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Restore original embeddings for all tokens EXCEPT the pseudo-token.
    # This ensures that only the new token's embedding is modified.
    with torch.no_grad():
        token_embeds.data[:placeholder_token_id] = original_embeds

    loss_val = loss.item()
    loss_history.append(loss_val)

    # Save checkpoint embedding.
    if (step + 1) in checkpoint_steps:
        checkpoint_embeds[step + 1] = token_embeds[placeholder_token_id].clone().detach()
        elapsed = time.time() - start_time
        print(f'  Step {step + 1}/{num_training_steps}: loss = {loss_val:.4f} ({elapsed:.1f}s elapsed)')

elapsed = time.time() - start_time
print(f'\nTraining complete in {elapsed:.1f}s.')
print(f'Final loss: {loss_history[-1]:.4f}')

<details>
<summary>Solution</summary>

**Why this approach works:** The training loop is identical to the DDPM loop from Learning to Denoise and LoRA Fine-Tuning. The only difference is the optimization target. We compute gradients through the entire frozen pipeline (CLIP encoder + U-Net) to update a single embedding row.

**TODO 1---Create the optimizer:**
```python
optimizer = torch.optim.AdamW(
    [text_encoder.get_input_embeddings().weight],
    lr=learning_rate,
)
```
The optimizer targets the full embedding weight tensor, but we manually restore all rows except the pseudo-token's row after each step. This is a simple approach for a short training run.

**TODO 2---One DDPM training step:**
```python
# a) VAE encode
with torch.no_grad():
    latents = vae.encode(image_tensor).latent_dist.mode() * vae.config.scaling_factor

# b) Sample random timestep
timesteps = torch.randint(
    0, noise_scheduler.config.num_train_timesteps, (1,), device=device
)

# c) Sample noise and add to latent
noise = torch.randn_like(latents)
noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

# d) Forward pass (text encoder needs gradients, so no torch.no_grad)
text_emb = text_encoder(input_ids)[0]
noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=text_emb).sample

# e) MSE loss
loss = F.mse_loss(noise_pred, noise)
```

**Common mistakes:**
- Wrapping the text encoder forward pass in `torch.no_grad()`. Gradients must flow through the CLIP encoder to reach the embedding table.
- Forgetting the VAE scaling factor (`* vae.config.scaling_factor`). The latents are scaled to have unit variance.
- Using `latent_dist.sample()` instead of `latent_dist.mode()`. Either works for training, but `mode()` is deterministic.

</details>

In [ ]:
# ============================================================
# Visualize training loss.
# ============================================================

if len(loss_history) > 0:
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(loss_history, color='cyan', alpha=0.3, linewidth=0.5, label='Per-step loss')

    # Smoothed loss (rolling average over 20 steps).
    window = 20
    if len(loss_history) >= window:
        smoothed = np.convolve(loss_history, np.ones(window)/window, mode='valid')
        ax.plot(range(window - 1, len(loss_history)), smoothed,
                color='cyan', linewidth=2, label=f'Smoothed ({window}-step avg)')

    for step in checkpoint_steps:
        if step <= len(loss_history):
            ax.axvline(x=step, color='#a855f7', linestyle='--', alpha=0.5)
            ax.annotate(f'Step {step}', (step, ax.get_ylim()[1]),
                       textcoords='offset points', xytext=(5, -15),
                       fontsize=8, color='#a855f7')

    ax.set_xlabel('Training Step', fontsize=11)
    ax.set_ylabel('MSE Loss', fontsize=11)
    ax.set_title('Textual Inversion Training Loss', fontsize=12)
    ax.legend(fontsize=10)
    plt.tight_layout()
    plt.show()
else:
    print('No training data to plot. Complete TODO 2 first.')

In [ ]:
# ============================================================
# Measure how far the embedding has moved from initialization.
# ============================================================

if len(checkpoint_embeds) > 0:
    init_embed = token_embeds[init_id].detach()

    print('Cosine similarity between checkpoints and initialization word:')
    print('-' * 55)
    for step, embed in sorted(checkpoint_embeds.items()):
        cos_sim = F.cosine_similarity(
            embed.unsqueeze(0), init_embed.unsqueeze(0)
        ).item()
        l2_dist = (embed - init_embed).norm().item()
        print(f'  Step {step:>4d}: cos_sim = {cos_sim:.4f}, L2 distance = {l2_dist:.4f}')

    print(f'\nThe embedding starts identical to "{initializer_token}" and drifts away')
    print(f'during training. It finds a NEW point in CLIP\'s continuous space---')
    print(f'not at any existing token\'s position, but nearby.')
else:
    print('No checkpoints saved. Complete the training loop first.')

In [ ]:
# ============================================================
# Generate images WITH and WITHOUT the pseudo-token.
# ============================================================
# This tests the "zero weight change" claim from the lesson:
# prompts without the pseudo-token should produce the same output
# as before training.

# Build a pipeline from the trained components.
# We switch to float16 for faster inference.
vae_f16 = vae.to(dtype=torch.float16)
unet_f16 = unet.to(dtype=torch.float16)
text_encoder_f16 = text_encoder.to(dtype=torch.float16)

pipeline = StableDiffusionPipeline(
    vae=vae_f16,
    text_encoder=text_encoder_f16,
    tokenizer=tokenizer,
    unet=unet_f16,
    scheduler=DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder='scheduler'),
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False,
)

# Generate with the pseudo-token.
prompt_with = f'a painting of a warrior in the style of {placeholder_token}'
prompt_without = 'a painting of a warrior in anime style'
prompt_generic = 'a beautiful sunset over the ocean'  # No pseudo-token at all

seed = 42
num_steps = 25

results = []
prompts_list = [prompt_with, prompt_without, prompt_generic]
titles = [
    f'With {placeholder_token}',
    'Without pseudo-token\n("anime style")',
    'Unrelated prompt\n("sunset")',
]

for p in prompts_list:
    generator = torch.Generator(device=device).manual_seed(seed)
    result = pipeline(
        prompt=p,
        num_inference_steps=num_steps,
        guidance_scale=7.5,
        generator=generator,
    ).images[0]
    results.append(result)

show_images(results, titles, figsize=(18, 6))

print(f'\nPrompts used:')
for p, t in zip(prompts_list, titles):
    print(f'  {t.split(chr(10))[0]:>30s}: "{p}"')
print(f'\nThe pseudo-token prompt should reflect the training concept (naruto style).')
print(f'The unrelated prompt ("sunset") verifies the model is UNCHANGED---it')
print(f'never routes through the new embedding, so the output is identical')
print(f'to what an unmodified model would produce.')

In [ ]:
# Save the learned embedding for later comparison in Exercise 4.
learned_embed_ti = token_embeds[placeholder_token_id].clone().detach().cpu()
ti_file_size = learned_embed_ti.numel() * 4  # 768 float32 values = 3072 bytes
print(f'Textual inversion embedding saved.')
print(f'  Shape: {list(learned_embed_ti.shape)}')
print(f'  Parameters: {learned_embed_ti.numel()}')
print(f'  File size: {ti_file_size:,} bytes ({ti_file_size / 1024:.1f} KB)')

# Record training time for comparison.
ti_training_time = elapsed
ti_training_steps = num_training_steps

# Clean up.
del pipeline, vae, unet, text_encoder, vae_f16, unet_f16, text_encoder_f16
cleanup()
print('Pipeline freed from VRAM.')

### What Just Happened

You trained a real textual inversion embedding and observed several key phenomena:

1. **The training loop is the same DDPM loop.** VAE encode, sample timestep, noise, predict noise, MSE loss, backprop. Identical to Learning to Denoise and LoRA Fine-Tuning. The only difference is the optimization target.

2. **The embedding drifts from its initialization.** Starting from "anime," the embedding gradually moves to a new point that encodes the visual characteristics of the training images. It is no longer at "anime"'s position---it is a genuinely new point in CLIP's continuous space.

3. **Style training previews the expressiveness limitation.** A single 768-dim vector can capture some visual flavor (color palette, general aesthetic) but struggles with the full complexity of an artistic style (spatial composition, lighting patterns, rendering techniques). If your results look vague or inconsistent, that is *expected*---this is the "one word ceiling" from the lesson. Exercise 4 will show how LoRA handles the same dataset.

4. **Zero weight change verification.** Prompts without the pseudo-token produce the same output as an unmodified model. The model is unchanged. Only the embedding vocabulary has grown by one entry.

5. **Tiny output file.** The learned embedding is 768 float32 values = ~3 KB. Compare this to a LoRA adapter file (2--50 MB). Three orders of magnitude smaller.

---

## Exercise 4: Compare Textual Inversion vs LoRA [Independent]

The lesson claimed that textual inversion is the most lightweight customization technique but less expressive than LoRA for complex styles. You experienced textual inversion firsthand in Exercise 3. Now, train a LoRA adapter on the same dataset and compare.

**Your task:**
1. Train a LoRA adapter on the same `lambdalabs/naruto-blip-captions` dataset (200--500 steps, same as the LoRA lesson's notebook)
2. Generate images from both textual inversion and LoRA using the same prompts and seeds
3. Compare: file size, training time, parameter count, and output quality
4. Write a brief comparison: which captures the concept better? Why?

**Key APIs:**
- LoRA setup: `from peft import LoraConfig, get_peft_model`
- LoRA targets: `['to_k', 'to_v', 'to_q', 'to_out.0']` (cross-attention projections)
- LoRA training loop: same DDPM loop, but optimizer targets LoRA params instead of embedding
- Generate with LoRA: load adapters into a StableDiffusionPipeline

**VRAM constraint:** Train LoRA in float16 with gradient accumulation if needed. Clean up between training and generation.

**What to compare:**
- Parameter count (768 vs LoRA's count)
- File size (~3 KB vs LoRA's file size)
- Training time for the same number of steps
- Output quality on simple prompts ("a warrior in [style]")
- Output quality on complex prompts ("a detailed landscape with dramatic lighting in [style]")

In [ ]:
# YOUR COMPARISON EXPERIMENT
#
# Step 1: Train a LoRA adapter on the same dataset.
#
# Load components:
#   tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
#   text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder='text_encoder', torch_dtype=torch.float32).to(device)
#   vae = AutoencoderKL.from_pretrained(model_id, subfolder='vae', torch_dtype=torch.float32).to(device)
#   unet = UNet2DConditionModel.from_pretrained(model_id, subfolder='unet', torch_dtype=torch.float32).to(device)
#   noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder='scheduler')
#
# Apply LoRA to U-Net:
#   from peft import LoraConfig, get_peft_model
#   lora_config = LoraConfig(
#       r=4,
#       lora_alpha=4,
#       target_modules=['to_k', 'to_v', 'to_q', 'to_out.0'],
#       lora_dropout=0.0,
#   )
#   unet = get_peft_model(unet, lora_config)
#   unet.print_trainable_parameters()  # See the parameter count
#
# Freeze everything except LoRA params:
#   vae.requires_grad_(False)
#   text_encoder.requires_grad_(False)
#   # LoRA params are already unfrozen by get_peft_model
#
# Training loop (same as Exercise 3, but optimizer targets LoRA params):
#   optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-4)
#   for step in range(num_training_steps):
#       image_tensor, _ = get_training_batch(dataset, step)
#       caption = 'a painting in naruto anime style'  # Use regular text, no pseudo-token
#       ... same DDPM steps as Exercise 3 ...
#       ... but text_encoder forward pass CAN use torch.no_grad() since we
#           are not training the embedding ...
#
# Step 2: Generate from both and compare.
#   Load the textual inversion embedding from Exercise 3 (learned_embed_ti).
#   Generate with matched prompts and seeds.
#
# Step 3: Compare file sizes, training times, parameter counts, and output quality.

print('Build your comparison experiment here.')
print('Train LoRA, generate from both, and compare.')
print()
print(f'Textual inversion stats from Exercise 3:')
print(f'  Parameters:    768')
print(f'  File size:     {ti_file_size:,} bytes ({ti_file_size / 1024:.1f} KB)')
print(f'  Training time: {ti_training_time:.1f}s ({ti_training_steps} steps)')

<details>
<summary>Solution</summary>

**Why this comparison matters:** The lesson claimed that textual inversion captures "what a concept looks like" (object identity, simple visual attributes) while LoRA captures "how images in a style should look" (spatial composition, color palettes, rendering techniques). This experiment lets you verify that claim with firsthand experience rather than taking the lesson's word for it.

```python
from peft import LoraConfig, get_peft_model
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import (AutoencoderKL, UNet2DConditionModel, DDPMScheduler,
                       DPMSolverMultistepScheduler, StableDiffusionPipeline)

# ============================================================
# PART 1: Train LoRA on the same dataset.
# ============================================================

print('Loading components for LoRA training...')
tokenizer_lora = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
text_encoder_lora = CLIPTextModel.from_pretrained(
    model_id, subfolder='text_encoder', torch_dtype=torch.float32
).to(device)
vae_lora = AutoencoderKL.from_pretrained(
    model_id, subfolder='vae', torch_dtype=torch.float32
).to(device)
unet_lora = UNet2DConditionModel.from_pretrained(
    model_id, subfolder='unet', torch_dtype=torch.float32
).to(device)
noise_scheduler_lora = DDPMScheduler.from_pretrained(model_id, subfolder='scheduler')

# Apply LoRA to U-Net cross-attention projections.
lora_config = LoraConfig(
    r=4,
    lora_alpha=4,
    target_modules=['to_k', 'to_v', 'to_q', 'to_out.0'],
    lora_dropout=0.0,
)
unet_lora = get_peft_model(unet_lora, lora_config)
unet_lora.print_trainable_parameters()

# Freeze non-LoRA components.
vae_lora.requires_grad_(False)
text_encoder_lora.requires_grad_(False)

# Training loop.
lora_training_steps = 300
optimizer_lora = torch.optim.AdamW(
    [p for p in unet_lora.parameters() if p.requires_grad],
    lr=1e-4,
)

lora_prompt = 'a painting in naruto anime style'
lora_loss_history = []

lora_text_input = tokenizer_lora(
    lora_prompt,
    padding='max_length',
    max_length=tokenizer_lora.model_max_length,
    truncation=True,
    return_tensors='pt',
)
lora_input_ids = lora_text_input.input_ids.to(device)

with torch.no_grad():
    lora_text_emb = text_encoder_lora(lora_input_ids)[0]

print(f'\nTraining LoRA for {lora_training_steps} steps...')
lora_start = time.time()

for step in range(lora_training_steps):
    idx = step % len(dataset)
    example = dataset[idx]
    image = example['image'].convert('RGB')
    image_tensor = image_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        latents = vae_lora.encode(image_tensor).latent_dist.mode() * vae_lora.config.scaling_factor

    timesteps = torch.randint(0, noise_scheduler_lora.config.num_train_timesteps, (1,), device=device)
    noise = torch.randn_like(latents)
    noisy_latents = noise_scheduler_lora.add_noise(latents, noise, timesteps)

    noise_pred = unet_lora(noisy_latents, timesteps, encoder_hidden_states=lora_text_emb).sample
    loss = F.mse_loss(noise_pred, noise)

    optimizer_lora.zero_grad()
    loss.backward()
    optimizer_lora.step()

    lora_loss_history.append(loss.item())
    if (step + 1) % 100 == 0:
        print(f'  Step {step + 1}/{lora_training_steps}: loss = {loss.item():.4f}')

lora_training_time = time.time() - lora_start
print(f'LoRA training complete in {lora_training_time:.1f}s.')

# Count LoRA parameters and estimate file size.
lora_param_count = sum(p.numel() for p in unet_lora.parameters() if p.requires_grad)
lora_file_size = lora_param_count * 4  # float32
print(f'LoRA parameters:  {lora_param_count:,}')
print(f'LoRA file size:   {lora_file_size:,} bytes ({lora_file_size / 1024:.1f} KB)')

# ============================================================
# PART 2: Generate from both and compare.
# ============================================================

# Build a pipeline with the LoRA adapter.
unet_lora_f16 = unet_lora.to(dtype=torch.float16)
text_encoder_lora_f16 = text_encoder_lora.to(dtype=torch.float16)
vae_lora_f16 = vae_lora.to(dtype=torch.float16)

lora_pipeline = StableDiffusionPipeline(
    vae=vae_lora_f16,
    text_encoder=text_encoder_lora_f16,
    tokenizer=tokenizer_lora,
    unet=unet_lora_f16,
    scheduler=DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder='scheduler'),
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False,
)

# Simple and complex prompts for comparison.
simple_prompt = 'a painting of a warrior'
complex_prompt = 'a detailed landscape with dramatic lighting and rain'

seed = 42
lora_results = []
for p in [simple_prompt, complex_prompt]:
    generator = torch.Generator(device=device).manual_seed(seed)
    result = lora_pipeline(
        prompt=f'{p} in naruto anime style',
        num_inference_steps=25,
        guidance_scale=7.5,
        generator=generator,
    ).images[0]
    lora_results.append(result)

del lora_pipeline, unet_lora, text_encoder_lora, vae_lora
del unet_lora_f16, text_encoder_lora_f16, vae_lora_f16
cleanup()

# Now generate from textual inversion for comparison.
# Reload the components with the textual inversion embedding.
tokenizer_ti = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
text_encoder_ti = CLIPTextModel.from_pretrained(
    model_id, subfolder='text_encoder', torch_dtype=torch.float16
).to(device)
vae_ti = AutoencoderKL.from_pretrained(model_id, subfolder='vae', torch_dtype=torch.float16).to(device)
unet_ti = UNet2DConditionModel.from_pretrained(model_id, subfolder='unet', torch_dtype=torch.float16).to(device)

# Re-add the pseudo-token and load the trained embedding.
tokenizer_ti.add_tokens([placeholder_token])
text_encoder_ti.resize_token_embeddings(len(tokenizer_ti))
ti_token_id = tokenizer_ti.convert_tokens_to_ids(placeholder_token)
with torch.no_grad():
    text_encoder_ti.get_input_embeddings().weight[ti_token_id] = learned_embed_ti.to(
        device=device, dtype=torch.float16
    )

ti_pipeline = StableDiffusionPipeline(
    vae=vae_ti,
    text_encoder=text_encoder_ti,
    tokenizer=tokenizer_ti,
    unet=unet_ti,
    scheduler=DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder='scheduler'),
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False,
)

ti_results = []
for p in [simple_prompt, complex_prompt]:
    generator = torch.Generator(device=device).manual_seed(seed)
    result = ti_pipeline(
        prompt=f'{p} in the style of {placeholder_token}',
        num_inference_steps=25,
        guidance_scale=7.5,
        generator=generator,
    ).images[0]
    ti_results.append(result)

del ti_pipeline, unet_ti, text_encoder_ti, vae_ti
cleanup()

# ============================================================
# PART 3: Display comparison.
# ============================================================

show_image_grid(
    [ti_results[0], lora_results[0], ti_results[1], lora_results[1]],
    ['TI: Simple Prompt', 'LoRA: Simple Prompt',
     'TI: Complex Prompt', 'LoRA: Complex Prompt'],
    nrows=2, ncols=2, figsize=(12, 12),
    suptitle='Textual Inversion vs LoRA on Same Dataset',
)

print(f'\n{"Metric":<25s} {"Textual Inversion":<25s} {"LoRA (r=4)":<25s}')
print(f'{"-" * 75}')
print(f'{"Parameters":<25s} {"768":<25s} {"{:,}".format(lora_param_count):<25s}')
print(f'{"File size":<25s} {"{:.1f} KB".format(ti_file_size / 1024):<25s} {"{:.1f} KB".format(lora_file_size / 1024):<25s}')
print(f'{"Training time":<25s} {"{:.1f}s".format(ti_training_time):<25s} {"{:.1f}s".format(lora_training_time):<25s}')
print(f'{"Training steps":<25s} {"{}".format(ti_training_steps):<25s} {"{}".format(lora_training_steps):<25s}')
print(f'{"What changes":<25s} {"Embedding table (+1 row)":<25s} {"U-Net cross-attn weights":<25s}')
print(f'{"Best for":<25s} {"Object identity, textures":<25s} {"Complex styles, spatial":<25s}')
```

**What to look for in the comparison:**
- On **simple prompts** (single subject), textual inversion and LoRA may produce similarly stylized results. Both can capture the general "naruto anime" visual flavor.
- On **complex prompts** (spatial composition, lighting), LoRA typically produces more consistent style transfer because it modifies cross-attention projections across the entire U-Net---each spatial resolution can encode different aspects of the style.
- Textual inversion's 768 parameters encode "what does this style look like as a concept descriptor." LoRA's thousands of parameters encode "how should each attention layer process features to produce this style."
- The file size difference is dramatic: ~3 KB vs hundreds of KB or MB. This matters for sharing and storage.

**Common mistakes:**
- Using different seeds for LoRA and textual inversion generation. Same seed + same prompt structure ensures a fair comparison.
- Forgetting to clean up between LoRA training and textual inversion generation. VRAM will overflow.
- Training LoRA with the text encoder in `torch.no_grad()` is correct here (unlike textual inversion, where gradients must reach the embedding).

</details>

---

## Key Takeaways

1. **CLIP's embedding table is a concrete data structure---49,408 rows of 768 floats.** Each row maps a token ID to a vector. Textual inversion adds one row and optimizes it. That is the entire technique: find the right point in a continuous, meaningful space.

2. **The training loop is the same DDPM loop you have seen four times.** VAE encode, sample timestep, noise, U-Net prediction, MSE loss, backprop. The only difference is what receives the gradient: one 768-float embedding vector instead of LoRA adapter matrices or full model weights.

3. **Frozen does not mean invisible to gradients.** Gradients flow through the entire frozen CLIP encoder and U-Net to reach the embedding table. The frozen layers are a pathway, not a wall. Multiple embedding rows receive gradients (via CLIP's self-attention), but the restore-original-rows pattern ensures only the pseudo-token's row is actually updated.

4. **Textual inversion is the most lightweight customization (768 parameters, ~3 KB file) but least expressive for complex styles.** LoRA's thousands of cross-attention parameters capture spatial composition, color palettes, and rendering techniques that a single embedding vector cannot encode. Choose the tool that matches the scope of the customization.

5. **Three knobs on three different parts of the pipeline.** LoRA changes the weights. Img2img/inpainting changes the inference process. Textual inversion changes the embeddings. You now understand all three.